## Retrieving

1. Start Docker Service
2. Start Docker configuration in `docker-compose.yaml` with `docker compose up`

In [ ]:
import os
import weaviate
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import StorageContext, VectorStoreIndex, get_response_synthesizer, Settings
from llama_index.llms.anthropic import Anthropic
from llama_index.embeddings.huggingface import HuggingFaceEmbedding



EMBEDDING_MODEL = 'aari1995/German_Semantic_STS_V2'

def save_file(file, text):
    f = open(file, "w")
    f.write(text)
    f.close()

In [ ]:
os.environ["ANTHROPIC_API_KEY"] = ""
tokenizer = Anthropic().tokenizer
llm = Anthropic(model="claude-3-haiku-20240307")
embedding_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL)
Settings.tokenizer = tokenizer
Settings.llm = llm
Settings.embed_model = embedding_model

In [ ]:
client = weaviate.Client("http://localhost:8080")
print(client.get_meta())

In [ ]:
vector_store = WeaviateVectorStore(weaviate_client = client, embed_model=embedding_model, llm=llm, index_name="BabyForum")
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, storage_context=storage_context)

In [ ]:
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
    llm=llm
)
response_synthesizer = get_response_synthesizer(response_mode="simple_summarize")

In [ ]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    # node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.8)],
)

In [ ]:
# from IPython.display import Markdown, display
# def display_prompt_dict(prompts_dict):
#     for k, p in prompts_dict.items():
#         text_md = f"**Prompt Key**: {k}<br>" f"**Text:** <br>"
#         display(Markdown(text_md))
#         print(p.get_template())
#         display(Markdown("<br><br>"))

In [ ]:
# prompts_dict = query_engine.get_prompts()
# display_prompt_dict(prompts_dict)

In [ ]:
from llama_index.core import PromptTemplate
PROMPT_TEMPLATE = """Du bist ein hilfreicher deutscher Assistent für junge Eltern, der Informationen aus Foren sammelt, um Eltern bei der Beantwortung von Fragen über ihre Kinder zu helfen.

            Verwende die folgenden Konversationen aus Elternforen, um den Nutzer über ein bestimmtes Thema zu helfen. Jede Zeile beginnt mit dem Namen des Benutzers, gefolgt von ":" und dann dem Kommentar, zum Beispiel so: "John: Bei mir ist es genauso."
            Verschiedene Konversationen können sich auf dasselbe Thema beziehen.
            <conversations>
            {context_str}
            </conversations>
            
            Wenn du die Unterhaltungen im Forum zitierst, gib bitte den Benutzernamen in deiner Antwort an. Hier sind Beispielw in <example> Tags:
            <example>
            Viele Nutzer sagen dass es normal ist Kinder nachts zu stillen. Cari234 sagt, z.B, dass sie täglich überfordert ist. Lomo2 hat gleiche Erfahrungen.
            </example>
            <example>
            Die Konversationen beinhalten keine Hinweise ob es normal ist, dass Kinder nachts Wachfenster haben.
            </example>

            Hier ist deine Aufgabe: Welche relevanten Informationen kannst du aus den oben genannten Gesprächen zu diesem Thema entnehmen? Das Thema ist unten in <question>-Tags:
            <question>
            {query_str}
            </question>

            Um deine Aufgabe zu erledigen, gehe die folgenden Schritte durch:
            1. Erstelle eine umfassende Zusammenfassung für jede Konversation in <conversation> Tags oben. Die Zusammenfassungen sollte alle wichtigen Punkte und Hauptgedanken des Originaltextes die sich auf diesem Aufgabe Thema beziehen abdecken und gleichzeitig die Informationen in einem prägnanten und leicht verständlichen Format zusammenfassen.
            Achte bitte darauf, dass die Zusammenfassung die Usernamen, relevante Details und Beispiele enthält, die die Hauptgedanken unterstützen, und vermeide unnötige Informationen oder Wiederholungen.
            2. Erledige deine Aufgabe auf der Grundlage der Zusammenfassungen von Schritt 1. Füge alle relevanten Informationen, Details und Beispiele ein die aus der Zusammenfassungen rauskommen und sich auf diesem Thema beziehen. Behalte die Antwort auf maximal 5 Sätze.

            Wenn du die Aufgabe erledigst, nenn bitte konkrete Beispiele und Tipps aus den Forendiskussionen und verallgemeinere die Details nicht. Wenn du die Antwort nicht weißt, sag einfach, dass du es nicht weißt.
            Fang deine Antwort mit "Das sagen andere Nutzer dazu:" an. Danach, gib die Zusammenfassungen von Schritt 1 aus als Bulletpoint-Liste aus. Für jede Konversation soll es ein Bulletpoint geben. Gib dann deine zusammenfassende Antwort gemäß Schritt 2 in eine neue Zeile ein.
            Hier sind Beispiele wie deine Antworten formattiert werden sollen, in <answer-example>-Tags:

            <answer-example>
            Das sagen andere Nutzer dazu:
            - jomda erklärt, dass Babys manchmal schreien, wenn ältere Geschwister versorgt werden müssen oder wenn das Baby nachts wach wird, weil der Betreuer kurz etwas erledigen muss.
            - Mami83 berichtet, dass ihr Baby auch Phasen hatte, in denen es sehr unruhig war und häufig nachts aufwachte. Caro34 bestätigt.

            Das sagen die Nutzer zusammengefasst: Häufiges nächtliches Aufwachen und Unruhe sind bei Babys oft normal und hängen mit deren Entwicklung und Bedürfnissen zusammen. Die Situation kann vorübergehend sehr herausfordernd sein, aber die Eltern müssen durchhalten, da es mit der Zeit wieder besser wird.
            </answer-example>
            <answer-example>
            Das sagen andere Nutzer dazu:
            - bilbo45 sagt, dass solche schwierigen Phasen normal sind und immer wieder kommen können, da sich Babys ständig weiterentwickeln. Sie ermutigt, durchzuhalten, da es mit der Zeit besser wird.
            - Micebwn beschreibt, dass ihr 9 Monate altes Baby seit Tagen nachts weinend aufwacht.

            Aus den Gesprächen geht hervor, dass bei den meisten Kindern dieser spezielle Test mit schwarzen und weißen Punkten, bei denen das Kind etwas Bestimmtes erkennen und zeigen soll, im Alter von 11 Monaten noch nicht funktioniert. Die Eltern berichten, dass ihre Kinder stattdessen eher versuchen, mit den Fingern in den Mund der Ärzte zu kommen oder generell eher an der Untersuchung interessiert sind als an dem Test.
            </answer-example>    

            Erledige jetzt bitte deine Aufgabe bezüglich des Themas von oben.
            """

qa_prompt_tmpl = PromptTemplate(PROMPT_TEMPLATE)
query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [ ]:
# prompts_dict = query_engine.get_prompts()
# display_prompt_dict(prompts_dict)

In [ ]:
answer1 = query_engine.query("Kind wacht stündlich auf")
for n in answer1.source_nodes:
    print(f"Node {n.node_id} with similarity score {n.score}:\n{n.text}\n\n\n")
print(answer1)

In [ ]:
answer2 = query_engine.query("Wer kümmert sich um das Kind nachts?")
for n in answer2.source_nodes:
    print(f"Node {n.node_id} with similarity score {n.score}:\n{n.text}\n\n\n")

In [ ]:
from pprint import pprint
answer3 = query_engine.query("Wann darf man mit kind ins Tropical Island?")

In [ ]:
pprint(str(answer3))

In [ ]:
print(os.environ.get('OPENAI_API_KEY'))
print(os.environ.get('ANTHROPIC_API_KEY'))

In [ ]:
# if client.collections.exists("ForumPost"):
#     client.collections.delete("ForumPost")
# client.collections.create(
#     "ForumPost",
#     vectorizer_config=Configure.Vectorizer.text2vec_transformers(),
#     generative_config=Configure.Generative.openai(model='gpt-3.5-turbo'),
#     properties=[
#         Property(name="body", data_type=DataType.TEXT),
#     ]
#     )

In [ ]:
# post1 = {'body': "Das ist ein langes Post über weinenende Kinder"}
# post2 = {'body': "Mein Baby isst nichts mehr tagsüber"}
# post3 = {'body': "Kind wacht Nachts weinend auf und schreit"}
# posts = client.collections.get("ForumPost")
# posts.data.insert(post1)
# posts.data.insert_many([post2, post3])

In [ ]:
# posts = client.collections.get("ForumPost")
# for item in posts.iterator(include_vector=True):
#     print(item.properties)
#     print(item.vector)


In [ ]:
# response = posts.query.near_text(
#         query="Baby weint",
#         return_metadata=wvc.query.MetadataQuery(distance=True),
#         limit=2
#     )

# for o in response.objects:
#     print(o.properties)
#     print(o.metadata)

In [ ]:
# response = posts.query.hybrid(
#         query="Baby weint",
#         return_metadata=wvc.query.MetadataQuery(distance=True, score=True, explain_score=True),
#         alpha=0.75,
#         limit=3
#     )

# for o in response.objects:
#     print(o.properties)
#     print(o.metadata)